# Install+Import

In [1]:
try: 
    import dicomsdl
except:
    %pip install /kaggle/input/rsna-2023-abdominal-trauma/dicomsdl-0.109.2-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl
    
import os 
import sys 
sys.path.append('/kaggle/input/rsna-2023-abdominal-trauma')

In [2]:
import numpy as np
import pandas as pd
from glob import glob
import torch
import torch.nn as nn
from timeit import default_timer as timer
from utils import get_seg_model,load_dicom_folder,resize3d,visualize,get_crops,LSKModel,BowelModel
from matplotlib import pyplot as plt
import gc
    
print('IMPORT OK!!!!')

2023-09-19 14:08:52.300466: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-19 14:08:53.077663: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


IMPORT OK!!!!


In [3]:
mode = 'local' #submit #local

if mode =='local':
    image_dir  = '/kaggle/input/rsna-2023-abdominal-trauma-detection/train_images'
    df_meta = pd.read_csv('/kaggle/input/rsna-2023-abdominal-trauma-detection/train_series_meta.csv')
    df = pd.read_csv('/kaggle/input/rsna-2023-abdominal-trauma-detection/train.csv') 

    
if mode =='submit':
    image_dir = '/kaggle/input/rsna-2023-abdominal-trauma-detection/test_images'
    df_meta = pd.read_csv('/kaggle/input/rsna-2023-abdominal-trauma-detection/test_series_meta.csv')
 
df_meta.sort_values(by=["patient_id","aortic_hu"],inplace=True)
df_meta.set_index(["patient_id","series_id"],inplace=True)

# Models

In [4]:
def load_seg_models(models_dir,kernel):
    models=[]
    for ifold in range(5):
        model=get_seg_model(pretrained=False)
        model.load_state_dict(torch.load(f"{models_dir}/{kernel}_fold{ifold}_best.pth"))
        models.append(model)
    return models

def load_lsk_models(models_dir,kernel,backbone):
    models=[]
    for ifold in range(5):
        model=LSKModel(backbone,pretrained=False)
        model.load_state_dict(torch.load(f"{models_dir}/{kernel}_fold{ifold}_best.pth"))
        models.append(model)
    return models


def load_bowel_models(models_dir,kernel,backbone):
    models=[]
    for ifold in range(5):
        model=BowelModel(backbone,pretrained=False)
        model.load_state_dict(torch.load(f"{models_dir}/{kernel}_fold{ifold}_best.pth"))
        models.append(model)
    return models



models_seg_dir="models_seg"
kernel_seg = 'timm3d_res18d_unet4b_128_128_128_dsv2_flip12_shift333p7_gd1p5_bs4_lr3e4_20x50ep'
seg_models=load_seg_models(models_seg_dir,kernel_seg)

models_lsk_dir="models_cls"
kernel_lsk = '0920_1bonev2_effv2s_224_15_6ch_augv2_mixupp5_drl3_rov1p2_bs8_lr23e5_eta23e6_50ep'
backbone_lsk1="tf_efficientnetv2_b2.in1k"
lsk_models=load_lsk_models(models_lsk_dir,kernel_lsk,backbone_lsk1)

models_bowel_dir="models_bowel"
kernel_bowel = '0920_1bonev2_effv2s_224_15_6ch_augv2_mixupp5_drl3_rov1p2_bs8_lr23e5_eta23e6_50ep'
backbone_bowel1="tf_efficientnetv2_b0.in1k"
bowel_models=load_bowel_models(models_bowel_dir,kernel_bowel,backbone_bowel1)


In [5]:
image_seg_size = [128, 128, 128]
def infer_mask(image_full,models,vis=False):
    image1=resize3d(image_full,image_seg_size)
    image=(image1/255.).astype(np.float32)
    with torch.no_grad():
        image=torch.from_numpy(image).cuda()
        image=image[None,None,...]
        pred_masks = []
        for model in models:
            model.cuda()
            pmask = model(image).squeeze()
            pmask=nn.Softmax(dim=0)(pmask)
            pred_masks.append(pmask)
    mask = torch.stack(pred_masks).mean(0)
    mask=torch.argmax(mask,dim=0)
    mask=mask.numpy(force=True).astype(np.uint8)
    if vis:
        visualize(image1,mask)
    return mask

In [6]:
def infer_lsk(image_crops,models,nslice=24):
    lsk_dict={0:"liver",1:"spleen",2:"kidney"}
    results={}

    with torch.no_grad():
        for iorgan in range(3): #liver,spleen,kidney
            inds=list(range(iorgan*nslice,(iorgan+1)*nslice))
            images=image_crops[inds,...]
            images=torch.from_numpy(images).cuda()
            images=images[None,...]
            mlogits=[]
            for model in models:
                model.cuda()
                logits = model(images).mean(dim=1).squeeze()
                logits = logits.numpy(force=True)
                mlogits.append(logits)
            results[lsk_dict[iorgan]]=np.mean(np.stack(mlogits),axis=0)
    return results
       

In [7]:
def infer_be(image_crops,models,start=3*24,nslice=128):
    results={}

    with torch.no_grad():
        inds=list(range(start,start+nslice))
        images=image_crops[inds,...]
        images=torch.from_numpy(images).cuda()
        images=images[None,...]
        mlogits=[]
        for model in models:
            model.cuda()
            logits = model(images)[1].squeeze()
            logits = logits.numpy(force=True)
            mlogits.append(logits)
        mlogits=np.mean(np.stack(mlogits),axis=0)
        results["bowel"]=mlogits[0]
        results["extravasation"]=mlogits[1]
    return results

In [8]:
def analyze_dicom(image_folder):
    image_full=load_dicom_folder(image_folder,None)
    mask=infer_mask(image_full,seg_models,vis=False)
    gc.collect()
    torch.cuda.empty_cache()
    cropped_images = get_crops(image_full,mask) 
    cropped_images=cropped_images.astype(np.float32)/255.
    lsk_logits=infer_lsk(cropped_images,lsk_models)
    print(lsk_logits)
    gc.collect()
    torch.cuda.empty_cache()
    be_logits=infer_be(cropped_images,bowel_models)
    print(be_logits)
    gc.collect()
    torch.cuda.empty_cache()
    
    
    

In [9]:
print(df_meta.head(10))

                      aortic_hu  incomplete_organ
patient_id series_id                             
19         14374         174.00                 0
26         41997         101.00                 0
           18881         255.00                 0
33         55570         105.00                 0
43         36714          54.75                 1
           24055         127.00                 0
96         39874         134.00                 0
122        21217         118.00                 0
           8218          194.00                 0
156        64005         206.00                 0


In [10]:
for patient_id, group in df_meta[:3].groupby(level=0):
    for ind,row in group.iterrows():
        patient_id,series_id=ind
        image_folder=f"{image_dir}/{int(patient_id)}/{int(series_id)}"
        print(image_folder)
        analyze_dicom(image_folder)


/kaggle/input/rsna-2023-abdominal-trauma-detection/train_images/19/14374
{'liver': array([ 1.905035 , -0.8535553, -2.6612365], dtype=float32), 'spleen': array([ 1.9559019, -0.860046 , -2.7232194], dtype=float32), 'kidney': array([ 1.8704159, -0.8377802, -2.608977 ], dtype=float32)}
[array(-2.027214, dtype=float32)]
[array(-2.027214, dtype=float32), array(-2.2425218, dtype=float32)]
[array(-2.027214, dtype=float32), array(-2.2425218, dtype=float32), array(-1.874557, dtype=float32)]
[array(-2.027214, dtype=float32), array(-2.2425218, dtype=float32), array(-1.874557, dtype=float32), array(-2.532045, dtype=float32)]
[array(-2.027214, dtype=float32), array(-2.2425218, dtype=float32), array(-1.874557, dtype=float32), array(-2.532045, dtype=float32), array(-2.3351157, dtype=float32)]
-2.2022908


╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│ /tmp/user/1000/ipykernel_1968519/3664039939.py:6 in <module>                              │
│                                                                                           │
│ [Errno 2] No such file or directory: '/tmp/user/1000/ipykernel_1968519/3664039939.py'     │
│                                                                                           │
│ /tmp/user/1000/ipykernel_1968519/722420137.py:12 in analyze_dicom                         │
│                                                                                           │
│ [Errno 2] No such file or directory: '/tmp/user/1000/ipykernel_1968519/722420137.py'      │
│                                                                                           │
│ /tmp/user/1000/ipykernel_1968519/3437643848.py:18 in infer_be                             │
│                                                                                           │
│ [Errno 2] No such file or directory: '/tmp/user/1000/ipykernel_1968519/3437643848.py'     │
╰───────────────────────────────────────────────────────────────────────────────────────────╯
IndexError: invalid index to scalar variable.